In [1]:
import pdblp
from xbbg import blp
import workdays
import time
import datetime
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import calendar

pd.set_option('display.max_columns', 70)
# https://smart-hint.com/python/style/
# https://www.salesanalytics.co.jp/datascience/datascience046/

%matplotlib inline
%run -m FX_Rank 

Currency Return as of 20230705 (vsUSD, BGN)


,Currency,1D,vs06/30,vs03/31,Carry
6,GBP,-0.1%,0.0%,3.0%,0.2%
1,NOK,-0.1%,0.6%,-1.9%,-1.4%
8,JPY,-0.1%,-0.2%,-8.2%,-1.2%
0,NZD,-0.2%,0.9%,-1.3%,-3.1%
7,CHF,-0.2%,-0.3%,1.8%,-1.8%
4,EUR,-0.2%,-0.5%,0.1%,-0.5%
5,CAD,-0.4%,-0.3%,1.8%,-0.2%
3,CNY,-0.5%,0.0%,-5.2%,-3.7%
2,AUD,-0.6%,-0.1%,-0.4%,-5.5%
9,SEK,-1.0%,-1.3%,-4.9%,-1.7%


Currency Return as of 20230705 (vsUSD, BGN)


,Currency,1D,vs06/30,vs03/31,Carry
6,GBP,-0.1%,0.0%,3.0%,0.2%
1,NOK,-0.1%,0.6%,-1.9%,-1.4%
8,JPY,-0.1%,-0.2%,-8.2%,-1.2%
0,NZD,-0.2%,0.9%,-1.3%,-3.1%
7,CHF,-0.2%,-0.3%,1.8%,-1.8%
4,EUR,-0.2%,-0.5%,0.1%,-0.5%
5,CAD,-0.4%,-0.3%,1.8%,-0.2%
3,CNY,-0.5%,0.0%,-5.2%,-3.7%
2,AUD,-0.6%,-0.1%,-0.4%,-5.5%
9,SEK,-1.0%,-1.3%,-4.9%,-1.7%


In [2]:
ODA = workdays.workday(date.today(), days=-1).strftime("%Y%m%d") #One day agp
TDA= workdays.workday(date.today(), days=-2)  #Two days ago

if TDA.month > 3 or (TDA.month == 3 and TDA.day == 31):
    YE = date(TDA.year, 3, 31).strftime("%Y%m%d")
else:
    YE = date(TDA.year - 1, 3, 31).strftime("%Y%m%d")
    
L = ["NZD","NOK","AUD","CNY","EUR","CAD","GBP","CHF","JPY","SEK"]
T = [i + "USD BGN Curncy" for i in L] 
df = blp.bdh(T, "px_last", YE, ODA, Days="W", Fill="P").reset_index()
df = df[["index"]+T]
df.columns = ["Date"] + L
df

,Date,NZD,NOK,AUD,CNY,EUR,CAD,GBP,CHF,JPY,SEK
0,2023-03-31,0.6258,0.0955,0.6685,0.14548,1.0839,0.7398,1.2337,1.0926,0.007527,0.096114
1,2023-04-03,0.6297,0.0971,0.6786,0.14540,1.0899,0.7442,1.2414,1.0959,0.007549,0.096403
2,2023-04-04,0.6313,0.0970,0.6751,0.14536,1.0953,0.7438,1.2501,1.1037,0.007592,0.097301
3,2023-04-05,0.6318,0.0958,0.6720,0.14536,1.0904,0.7431,1.2462,1.1028,0.007615,0.096124
4,2023-04-06,0.6243,0.0957,0.6672,0.14550,1.0922,0.7411,1.2440,1.1056,0.007588,0.095894
...,...,...,...,...,...,...,...,...,...,...,...
64,2023-06-29,0.6070,0.0926,0.6616,0.13798,1.0865,0.7546,1.2613,1.1118,0.006908,0.091934
65,2023-06-30,0.6125,0.0931,0.6664,0.13786,1.0909,0.7552,1.2703,1.1165,0.006930,0.092613
66,2023-07-03,0.6153,0.0935,0.6673,0.13799,1.0912,0.7547,1.2693,1.1158,0.006912,0.092184
67,2023-07-04,0.6191,0.0938,0.6692,0.13858,1.0879,0.7562,1.2713,1.1148,0.006922,0.092331


In [3]:
T0 = [i + " BGN Curncy" for i in L]
T1 = [i + "1M BGN Curncy" for i in L]
T1 = [x.replace("CNY", "CNH") for x in T1]

BDP0 = blp.bdp(tickers=T0, flds=["px_last","fwd_scale","is_pct_chg_app_base_crncy"]).loc[T0,:]
BDP1 = blp.bdp(tickers=T1, flds=["px_last"]).loc[T1,:]

BDP = BDP0.reset_index(drop=True)
BDP.insert(0, "Curncy", L)
BDP.insert(2, "frd_point", BDP1["px_last"].tolist())
BDP.insert(3, "Carry", 0)
BDP["Carry"] = (1 + BDP["frd_point"] / BDP["px_last"] / 10 ** BDP["fwd_scale"] ) ** 12 - 1

BDP

,Curncy,px_last,frd_point,Carry,fwd_scale,is_pct_chg_app_base_crncy
0,NZD,0.6186,-0.86,-0.001667,4,N
1,NOK,10.6829,-122.08,-0.013627,4,Y
2,AUD,0.6656,6.60,0.011964,4,N
3,CNY,7.2487,-187.00,-0.030522,4,Y
4,EUR,1.0845,16.49,0.0184,4,N
5,CAD,1.3294,-5.98,-0.005385,4,Y
6,GBP,1.2709,2.40,0.002268,4,N
7,CHF,0.8981,-27.92,-0.036674,4,Y
8,JPY,143.75,-67.00,-0.054519,2,Y
9,SEK,10.9666,-153.35,-0.016652,4,Y


In [4]:
TDA= workdays.workday(date.today(), days=-2) #Two days ago
ME = (date(TDA.year, TDA.month, 1) + relativedelta(months=-1, day=31)).strftime("%Y/%m/%d") #Month End
YE = date(TDA.year, 3, 31) if TDA.month > 3 or (TDA.month == 3 and TDA.day == 31) else date(TDA.year - 1, 3, 31) #Year End
YE = YE.strftime("%Y/%m/%d")
TDA = TDA.strftime("%Y/%m/%d")  

C = ["TDA","ME","YE"]
FX = pd.DataFrame(L, columns=["Curncy"])

tmp = df.loc[len(df)-1, L]

for i in C:
    exec(f"d = {i}")
    FX[i] = (tmp / df.loc[df["Date"]==d, L] - 1).values[0]

FX.sort_values(by=C[0], ascending=False, inplace=True)
FX["Carry"] = np.where(BDP["is_pct_chg_app_base_crncy"] == "Y", BDP["Carry"], -BDP["Carry"])

C = ["Currency","1D","vs"+ME[5:],"vs"+YE[5:],"Carry"]
FX.columns= C

print("Currency Return as of "+ODA+"  (vsUSD, BGN)")
display(FX.style\
        .bar(subset=[C[1]],align='mid',color=["pink","lightblue"])\
        .bar(subset=[C[2]],align='mid',color=["pink","lightblue"])\
        .bar(subset=[C[3]],align='mid',color=["pink","lightblue"])\
        .bar(subset=[C[4]],align='mid',color=["pink","lightblue"])\
        .format({C[1]: '{:.1%}'})\
        .format({C[2]: '{:.1%}'})\
        .format({C[3]: '{:.1%}'})\
        .format({C[4]: '{:.1%}'})\
       )

Currency Return as of 20230705  (vsUSD, BGN)


,Currency,1D,vs06/30,vs03/31,Carry
6,GBP,-0.1%,0.0%,3.0%,0.2%
1,NOK,-0.1%,0.6%,-1.9%,-1.4%
8,JPY,-0.1%,-0.2%,-8.2%,-1.2%
0,NZD,-0.2%,0.9%,-1.3%,-3.1%
7,CHF,-0.2%,-0.3%,1.8%,-1.8%
4,EUR,-0.2%,-0.5%,0.1%,-0.5%
5,CAD,-0.4%,-0.3%,1.8%,-0.2%
3,CNY,-0.5%,0.0%,-5.2%,-3.7%
2,AUD,-0.6%,-0.1%,-0.4%,-5.5%
9,SEK,-1.0%,-1.3%,-4.9%,-1.7%
